# Hoy es 18 de agosto y se terminaron de bajar los comentarios de ML

In [3]:
import os
import pandas as pd
import numpy as np
import string
import re
#import fasttext

In [5]:
# Leo los datos:
print('leyendo los datos...')
df = pd.concat([pd.read_csv('./parts/{}'.format(filename)) for filename in os.listdir('./parts/')[:3]])
df.head(5)

leyendo los datos...


,prod_id,cat_id,review_id,country,prod_title,reviewer_id,review_date,review_status,review_title,review_content,review_rate,review_likes,review_dislikes
0,MLA739744701,MLA432788,69293174,MLA,Granza - Grava Para Peceras Acuarios X 3 Kilos...,183875093,2020-07-09T11:55:12.000-04:00,published,Muy bueno,Todo como debe ser. Lo recomiendo.,4,0,0
1,MLA739744701,MLA432788,56800912,MLA,Granza - Grava Para Peceras Acuarios X 3 Kilos...,175171274,2020-02-27T12:45:24.000-04:00,published,Mis peces agradecen mucho sus piedritas :D,Buenísimo el producto! no traten de comerlo ig...,5,0,0
2,MLA739744701,MLA432788,52758737,MLA,Granza - Grava Para Peceras Acuarios X 3 Kilos...,135187285,2020-01-02T14:44:52.000-04:00,published,Excelente,"Excelente, las uso para macetas de suculentas,...",5,0,0
3,MLA751596868,MLA432788,66792680,MLA,Grava Negra Acuarios-hidroponia Por Litro (nuevo),115756469,2020-06-21T01:30:10.000-04:00,published,Excelente,"Muy buena grava, queda muy lindo en acuarios y...",5,0,0
4,MLA641214779,MLA432788,22434109,MLA,Arena De Marmol Negro Super Fina Grava Acuario...,5999479,2018-01-19T16:42:07.000-04:00,published,Excelente,Muy bueno para peceras de ciclidos.,5,0,0


In [6]:
len(df)

802

In [4]:
# Elimino los duplicados:
print('eliminando duplicados...')
df = df.drop_duplicates(subset=['review_content']).reset_index(drop=True)

# Cambio todos los espacios por espacios simples:
print('limpiando espacios...')
df['review_content'] = df['review_content'].str.replace(r'\s+',' ',regex=True)

eliminando duplicados...
limpiando espacios...


In [5]:
len(df)

3827478

In [6]:
# Defino los caracteres con los que me voy a quedar y elimino 
# los comentarios que tienen caracteres que no van:
print('eliminando comentarios con caracteres raros...')
non_ascii = 'áàâãäÁÀÂÃÄéèêëÉÈÊẼËíìîĩïÍÌÎĨÏóòôõöÓÒÔÕÖúùûũüÚÙÛŨÜñÑçÇ'
chars_vocab = string.digits + string.ascii_letters + string.punctuation + string.whitespace + non_ascii
all_chars = df['review_content'].str.split('').explode().value_counts().index
chars_to_remove = list(set(all_chars) - set(chars_vocab) - {''})
chars_to_remove_regex = '|'.join(sorted([re.escape(c) for c in chars_to_remove]))
df = df[-df['review_content'].str.contains(chars_to_remove_regex,regex=True,na=True)].reset_index(drop=True)

# Separo por país:
print('separando por país...')
df_por = df[df['country'] == 'MLB'].reset_index(drop=True)
df_esp = df[df['country'] != 'MLB'].reset_index(drop=True)

# Elimino comentarios en otros idiomas:
model_predict = fasttext.load_model('./lid.176.bin').predict
def detect_language(s):
    l, _ = next(zip(*model_predict(s, k=1)))
    return l
print('eliminando por idioma...')
df_por = df_por[df_por['review_content'].apply(detect_language) == '__label__pt'].reset_index(drop=True)
print('eliminando por idioma...')
df_esp = df_esp[df_esp['review_content'].apply(detect_language) == '__label__es'].reset_index(drop=True)

eliminando comentarios con caracteres raros...
separando por país...
eliminando por idioma...
eliminando por idioma...


In [12]:
len(df[df['country'] != 'MLB'])

2477372

In [7]:
len(df_por), len(df_esp)

(1317260, 2462107)

Ver longitud de comentarios

In [8]:
max_comments = 30 # cantidad máxima de comentarios por producto
n = 30000 # cantidad de comentarios por calificación
    
def filter_by_importance(df,max_comments,n):

    # Pesos de cada característica
    w_rate = 1.
    w_like = .5
    w_dislike = .1
    w_country = 1.

    # Puntuación parcial de cada característica
    score_rate = 1/df['review_rate']
    score_rate = (score_rate-min(score_rate))/(max(score_rate)-min(score_rate))
    score_like = (df['review_likes']-min(df['review_likes']))/(max(df['review_likes'])-min(df['review_likes']))
    score_dislike = (df['review_dislikes']-min(df['review_dislikes']))/(max(df['review_dislikes'])-min(df['review_dislikes']))
    countries = df['country'].value_counts()
    total_counts = sum(countries)
    country2score = {country: countries[country] / total_counts for country in countries.index}
    score_country = df['country'].map(country2score)
    score_country = (score_country-min(score_country))/(max(score_country)-min(score_country))
    score_country.fillna(0,inplace=True)

    # Puntuación total
    total_score = w_rate*score_rate + w_like*score_like + w_dislike*score_dislike - w_country*score_country

    # Reordeno los comentarios por importancia y me quedo con,
    # a lo sumo, max_comments por comentario
    indices = total_score.sort_values(ascending=False).index
    df = df.loc[indices,:].reset_index(drop=True)
    df = df.groupby('prod_id').head(max_comments)

    print(df['review_rate'].value_counts())
    # Muestreamos aleatoriamente para quedarnos con n comentarios por rate, y mezclamos:
    random_state = 1234
    random_state = np.random.RandomState(random_state)
    df = df.iloc[np.hstack([random_state.choice(arr, n, replace=False) 
                           for arr in df.groupby('review_rate').indices.values()]),:].reset_index(drop=True)
    df = df.iloc[random_state.permutation(len(df)),:].reset_index(drop=True)

    return df

df_esp_filtered = filter_by_importance(df_esp,max_comments,n)
df_por_filtered = filter_by_importance(df_por,max_comments,n)

print('guardando resultados...')
df_esp_filtered.to_csv('data_esp.csv',index=False)
df_por_filtered.to_csv('data_por.csv',index=False)

5.0    1371660
4.0     395016
3.0     132461
1.0      62504
2.0      44498
Name: review_rate, dtype: int64
5.0    652203
4.0    174461
3.0     72922
1.0     46556
2.0     30623
Name: review_rate, dtype: int64
guardando resultados...


# TO DO:

* Terminar de filtrar los comentarios:
    * Ver longitud de los comentarios
    * Revisar diferencias entre rate y calificación
    * ...
* Trabajar en paralelo con un dataset en inglés: 
    * juntar un dataset parecido (amazon?) y probar los modelos entrenados para uno y para otro. 
    * Probar performance de un modelo multilingüe
    * ...
    
* Hacer el github y empezar a subir archivos.